In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd

from utils.makepath import makepath

In [3]:
root_dir = makepath("..", "..")

os.listdir(root_dir)

['scripts',
 'requirements.txt',
 'mri.egg-info',
 'venv',
 'README.md',
 'pretrained',
 'figures',
 'config',
 'LICENSE',
 'notebooks',
 'utils',
 'networks',
 'tmp',
 '.gitignore',
 'train_mri_example.ipynb',
 'gradops',
 'pyproject.toml',
 'results',
 'data',
 'pdhg',
 'model_10_20-15_48.pt',
 '.gitattributes',
 'model_10_20-15_47.pt',
 'wandb',
 'encoding_objects',
 '.git',
 'model_10_20-15_27.pt']

In [4]:
results_path = makepath(root_dir, "results", "test_set")

os.listdir(results_path)

['scalar_tgv',
 'u_tgv',
 'new_u_tv',
 'u_tv',
 'scalar_tv',
 'zero_filled',
 'new_u_tgv']

## Scalar manual search

In [5]:
# model = "scalar_tgv"
model = "scalar_tv"
best_metric = "SSIM"
for acc_R in [4, 8]:
    for sigma in ["0_05", "0_1", "0_2"]:
        # acc_R, sigma, datetime = 4, "0_05", "21_01_59"
        # acc_R, sigma, datetime = 4, "0_1", "21_03_20"
        # csv_file = f"test-abs-scalar_tgv-best_SSIM-R_{acc_R}-sigma_{sigma}-lambda0_w_0_to_0_2-lambda1_v_0_to_0_2-ternary_iters_20-dt_{datetime}.csv"
        for file in os.listdir(makepath(results_path, model)):
            if f"test-abs-{model}-best_{best_metric}-R_{acc_R}-sigma_{sigma}" in file and file.endswith(".csv"):
                csv_file = file
                break
        df = pd.read_csv(makepath(results_path, model, csv_file))
        # print(df.head())
        print(f"R = {acc_R}, sigma = {sigma}")
        print(f"PSNR: {df['PSNR'].mean():.2f} ± {df['PSNR'].std():.2f}")
        print(f"SSIM: {df['SSIM'].mean():.4f} ± {df['SSIM'].std():.3f}")
        print()

R = 4, sigma = 0_05
PSNR: 27.21 ± 1.81
SSIM: 0.7134 ± 0.088

R = 4, sigma = 0_1
PSNR: 27.15 ± 1.82
SSIM: 0.7144 ± 0.090

R = 4, sigma = 0_2
PSNR: 26.91 ± 1.79
SSIM: 0.7096 ± 0.088

R = 8, sigma = 0_05
PSNR: 23.92 ± 1.71
SSIM: 0.6262 ± 0.085

R = 8, sigma = 0_1
PSNR: 23.87 ± 1.71
SSIM: 0.6264 ± 0.085

R = 8, sigma = 0_2
PSNR: 23.77 ± 1.71
SSIM: 0.6249 ± 0.085



In [6]:
csv_path = makepath(results_path, model, csv_file)

df = pd.read_csv(csv_path)
df.head()

,id,PSNR,SSIM,lambda
0,1,24.973553,0.685744,0.014551
1,2,23.652971,0.560019,0.189642
2,3,26.778214,0.603287,0.155243
3,4,23.680239,0.672584,0.012823
4,5,22.243095,0.625197,0.022588


In [7]:
df.mean()

id        151.500000
PSNR       23.771406
SSIM        0.624861
lambda      0.069194
dtype: float64

In [8]:
df.std()

id        87.324109
PSNR       1.712432
SSIM       0.085098
lambda     0.061887
dtype: float64

## Pre-trained models

In [16]:
# model = "u_tgv"
model = "new_u_tgv"
df_results = pd.DataFrame()
for acc_R in [4, 8]:
    for sigma in ["0_05", "0_10", "0_20"]:
        csv_file = makepath(
            f"test-abs-R_{acc_R}-sigma_{sigma}",
            f"test-abs-R_{acc_R}-sigma_{sigma}_intermediate_metrics.csv",
        )
        df = pd.read_csv(makepath(results_path, model, csv_file))
        # print(df.head())
        print(f"R = {acc_R}, sigma = {sigma}")
        print(f"PSNR: {df['psnr'].mean():.2f} ± {df['psnr'].std():.2f}")
        print(f"SSIM: {df['ssim'].mean():.4f} ± {df['ssim'].std():.3f}")
        print()
        df_results = pd.concat([
            df_results,
            pd.DataFrame({
                "R": [acc_R],
                "sigma": [sigma],
                "PSNR": [f"{df['psnr'].mean():.2f} ± {df['psnr'].std():.2f}"],
                "SSIM": [f"{df['ssim'].mean():.4f} ± {df['ssim'].std():.3f}"],
            }),
        ])

R = 4, sigma = 0_05
PSNR: 31.17 ± 2.11
SSIM: 0.7855 ± 0.078

R = 4, sigma = 0_10
PSNR: 31.05 ± 2.11
SSIM: 0.7867 ± 0.077

R = 4, sigma = 0_20
PSNR: 30.52 ± 2.11
SSIM: 0.7866 ± 0.073

R = 8, sigma = 0_05
PSNR: 27.34 ± 1.95
SSIM: 0.7208 ± 0.076

R = 8, sigma = 0_10
PSNR: 27.31 ± 1.99
SSIM: 0.7240 ± 0.074

R = 8, sigma = 0_20
PSNR: 26.94 ± 2.00
SSIM: 0.7178 ± 0.071



In [17]:
print(df_results.to_latex())

\begin{tabular}{lrlll}
\toprule
 & R & sigma & PSNR & SSIM \\
\midrule
0 & 4 & 0_05 & 31.17 ± 2.11 & 0.7855 ± 0.078 \\
0 & 4 & 0_10 & 31.05 ± 2.11 & 0.7867 ± 0.077 \\
0 & 4 & 0_20 & 30.52 ± 2.11 & 0.7866 ± 0.073 \\
0 & 8 & 0_05 & 27.34 ± 1.95 & 0.7208 ± 0.076 \\
0 & 8 & 0_10 & 27.31 ± 1.99 & 0.7240 ± 0.074 \\
0 & 8 & 0_20 & 26.94 ± 2.00 & 0.7178 ± 0.071 \\
\bottomrule
\end{tabular}



In [11]:
# model = "u_tv"
model = "new_u_tv"
for acc_R in [4, 8]:
    for sigma in ["0_05", "0_10", "0_20"]:
        csv_file = makepath(
            f"test-abs-R_{acc_R}-sigma_{sigma}",
            f"test-abs-R_{acc_R}-sigma_{sigma}_intermediate_metrics.csv",
        )
        df = pd.read_csv(makepath(results_path, model, csv_file))
        # print(df.head())
        print(f"R = {acc_R}, sigma = {sigma}")
        print(f"PSNR: {df['psnr'].mean():.2f} ± {df['psnr'].std():.2f}")
        print(f"SSIM: {df['ssim'].mean():.4f} ± {df['ssim'].std():.3f}")
        print()

R = 4, sigma = 0_05
PSNR: 31.88 ± 2.21
SSIM: 0.6895 ± 0.094

R = 4, sigma = 0_10
PSNR: 31.63 ± 2.18
SSIM: 0.6808 ± 0.094

R = 4, sigma = 0_20
PSNR: 30.79 ± 2.13
SSIM: 0.6878 ± 0.092

R = 8, sigma = 0_05
PSNR: 26.99 ± 1.93
SSIM: 0.6676 ± 0.084

R = 8, sigma = 0_10
PSNR: 27.31 ± 2.05
SSIM: 0.6593 ± 0.085

R = 8, sigma = 0_20
PSNR: 27.17 ± 2.13
SSIM: 0.6663 ± 0.084

